# Get emails sent
Save each email to a folder with both HTML and TEXT versions along with a .csv file of all recipeints


## Configuration
We need some login details, we can pull in a config file or set some variables.
https://towardsdatascience.com/keeping-credentials-safe-in-jupyter-notebooks-fbd215a8e311

Provide a file name in "config_file" or provide values in the individual variables. The config file will override the listed variables.

If we have a list of export IDs we wish to use, we can use those.

In [ ]:
cfg = {'pardot_api_creds':{}} # dictionary of configruation details, ignore this here

#provide a value for 

config_file = "pardotlogin.ipynb" 

# OR all of the below

cfg['pardot_api_creds']['PardotOath_Client_id'] = ""
cfg['pardot_api_creds']['PardotOath_client_secret'] = ""
cfg['pardot_api_creds']['PardotOath_username'] = ""
cfg['pardot_api_creds']['PardotOath_password'] = ""
cfg['pardot_api_creds']['PardotOath_token'] = ""

cfg['pardot_api_creds']['PardotURL'] = ""
cfg['pardot_api_creds']['apiversion'] = ""



exportids=[1276, 1278, 1280] # list of ids to use, leave blank to autocreate  [1276, 1278, 1280]


## Code Setup
Making sure we have login details, getting a Pardot API library, logging in, and also setting some number formatting so our IDs read well.

In [ ]:
# get the login details

if config_file:
    """Ovrride any manually entered config above"""
    %run $config_file


# Pull in a Pardot API common library
%run ./PardotAPI.ipynb


#Log into Pardot
access_token = PardotLogin()

pd.options.display.float_format = '{:.0f}'.format # Get rid of adding trailing zeros to our numbers

## Create an export of all the Emails Sent
Because the export has a limit of 1 year of data per export, we get the Pardot account creation date and create an export file for every year the account has existed. Easy to forget to get all the years when a new year comes to be.


In [ ]:
PardotAccount = PardotAccountRead()

if not exportids:
    years = range(int(PardotAccount['account']['created_at'][ 0 : 4 ]),2021+1) #stops before the specified number
    years_list = list(years)

    for i in years_list:

        url = cfg['pardot_api_creds']['PardotURL'] +"/api/export/version/"+ str(cfg['pardot_api_creds']['apiversion']) +"/do/create?format=json"

        payload = json.dumps({
          "object": "visitorActivity",
          "procedure": {
            "name": "filter_by_created_at",
            "arguments": {
              "created_after": str(i)+"-01-01 00:00:00",
              "created_before": str(i)+"-12-31 23:59:59"
            }
          }
        })
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer ' + access_token,
          'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID'],
        }

        response = requests.request("POST", url, headers=headers, data=payload)
        response_dict = json.loads(response.text)
        exportids.append(response_dict['export']['id'])

        print(response.text)

In [ ]:
emailstemplates

### Read the export
Loop the export results and get data.
Note that there is a number of commented out options. We often need to get different things, and it's easy to uncomment the thing that we want, and comment out those that we don't so we don't need to look eveyrthing up again.

In [ ]:
df = pd.DataFrame() # Pandas Datafame library  - import pandas as pd 

filelimit = None #1 # None or an int - This is used to limit impact while extending code so we aren't sucking in ALL the data everytime
filecount = 0

for exportid in exportids:
    print (exportid)
    FileURLs = PardotGetExportFilesURLList()    
    #print(FileURLs)
    for fileURL in FileURLs:
        print(fileURL)
        temp = PardotDownloadFileURL(fileURL)
        #temp = temp[temp.email_template_id.notnull()]
        #temp = temp[temp.email_id.notnull()]
        
        
        if '<?xml version="1.0" encoding="UTF-8"?>' in temp.columns:
            temp.drop(columns=['custom_redirect_id', 'file_id', 'form_id', 'form_handler_id','landing_page_id','multivariate_test_variation_id', 'opportunity_id','paid_search_ad_id','site_search_query_id','visit_id','visitor_id', 'visitor_page_view_id','<?xml version="1.0" encoding="UTF-8"?>'], inplace=True)
        else:
            temp.drop(columns=['custom_redirect_id', 'file_id', 'form_id', 'form_handler_id','landing_page_id','multivariate_test_variation_id', 'opportunity_id','paid_search_ad_id','site_search_query_id','visit_id','visitor_id', 'visitor_page_view_id'], inplace=True)

        df = df.append(temp)
  
        filecount +=1
        if filelimit:
            if filelimit <= filecount:
                break
    if filelimit:
        if filelimit <= filecount:
            break

df.shape
#temp

## Sort the data
Sort by created_at so we create the folder on the first send date of the email

In [ ]:
# sort the dataframe
df.sort_values(by=['created_at'],  inplace=True)
# set the index to be this and don't drop
df.set_index(keys=['created_at'], drop=False,inplace=True)

#df.head(10)

## Get just the List Email IDs

In [ ]:
# get a list of email template IDs
emails=df['list_email_id'].unique().tolist()
emailstemplates=df[df['list_email_id'].isna()]['email_template_id'].unique().tolist()

#emailstemplates

In [ ]:
import matplotlib.pyplot as plt
# if using a Jupyter notebook, include:
%matplotlib inline

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = [ str(Prospects.count())+' Prospects', str(Leads.count())+' Leads', str(Contacts.count())+' Contacts']
sizes = [Prospects.count(), Leads.count() , Contacts.count()]


fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%')
ax.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
ax.set_title('Bounced Emails by Record Types')


plt.show()


## Safe file folder creation
Create folders if we need to automatically

In [ ]:
# Taken from https://stackoverflow.com/a/600612/119527
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise

def safe_open_w(path):
    ''' Open "path" for writing, creating any parent directories as needed.
    '''
    mkdir_p(os.path.dirname(path))
    return open(path, 'w', encoding='utf-8')

## Write out data
Write different fields for each type.

In [ ]:
dt = date.today().strftime( '%Y-%m-%d')
AccountName = PardotAccount['account']['company'].replace(" ", "")


for email in emails:
    if not pd.isna(email):
        
        #pull the created date from the first record on list
        dt = datetime.strptime(df.loc[df.list_email_id==email].iloc[0]['created_at'], '%Y-%m-%d %H:%M:%S')
        
        with safe_open_w(AccountName+'/listemails/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(email))+"/"+str(int(email))+".csv") as f:
            df.loc[df.list_email_id==email].to_csv(f, sep=",", float_format='%.2f',index=False, line_terminator='\n',encoding='utf-8')

            

        response_dict = PardotEmailRead()
        
        if response_dict.get('email'):
            if response_dict['email']['message']['html']:
 
                if response_dict['email']['message']['html']:
                    with safe_open_w((AccountName+'/listemails/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(email))+"/"+str(int(email))+".html") as f:
                        f.write(response_dict['email']['message']['html'])
                if response_dict['email']['message']['text']:
                    with safe_open_w((AccountName+'/listemails/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(email))+"/"+str(int(email))+".text") as f:
                        f.write(response_dict['email']['message']['text'])
            else:
                #print(response.text)
                #print(df.loc[df.list_email_id==email].iloc[0]['email_template_id'])

                if not pd.isna(df.loc[df.list_email_id==email].iloc[0]['email_template_id']):
                    response_dict = PardotEmailtemplateRead()       
                    if response_dict.get('emailTemplate'):
                        if response_dict['emailTemplate']['htmlMessage']:
                            with safe_open_w((AccountName+'/listemails/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(email))+"/"+str(int(email))+".html") as f:
                                f.write(response_dict['emailTemplate']['htmlMessage'])
                        if response_dict['emailTemplate']['textMessage']:
                            with safe_open_w((AccountName+'/listemails/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(email))+"/"+str(int(email))+".text") as f:
                                f.write(response_dict['emailTemplate']['textMessage'])
                    else:
                        print("missing emailtemplate body")
                        print(email)
                        print(response_dict)

                else:
                    print(email)
                    print(df.loc[df.list_email_id==email].iloc[0]['email_template_id'])
                        
        else:
            print("missing email body")
            print(email)
            print(response_dict)


In [ ]:
for emailtemplate in emailstemplates:
    if not pd.isna(emailtemplate):
        #pull the created date from the first record on list
        dt = datetime.strptime(df.loc[df.email_template_id==emailtemplate].iloc[0]['created_at'], '%Y-%m-%d %H:%M:%S')

        #print(str(int(emailtemplate)))
        #print(df.loc[df.email_template_id==emailtemplate])
        #print(df.loc[df.email_template_id==emailtemplate].iloc[0]['email_template_id'])

        
        with safe_open_w(AccountName+'/emailtemplates/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(emailtemplate))+"/"+str(int(emailtemplate))+".csv") as f:
            df.loc[df.email_template_id==emailtemplate].to_csv(f, sep=",", float_format='%.2f',index=False, line_terminator='\n',encoding='utf-8')

            
        import csv

        with open(AccountName+'/emailtemplates/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(emailtemplate))+"/"+str(int(emailtemplate))+".csv", 'r', encoding="utf8") as fin, open('fout.csv', 'w', newline='') as fout:

            # define reader and writer objects
            reader = csv.reader(fin, skipinitialspace=True)
            writer = csv.writer(fout, delimiter=',')

            # write headers
            writer.writerow(next(reader))

            # iterate and write rows based on condition
            for i in reader:
                #print(i)
                if not i[7]:
                    writer.writerow(i)
            
          
        from shutil import move
        move('fout.csv', AccountName+'/emailtemplates/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(emailtemplate))+"/"+str(int(emailtemplate))+".csv")
        
        #if not pd.isna(df.loc[df.email_template_id==emailtemplate].iloc[0]['email_template_id']):
        if not pd.isna(emailtemplate):

            response_dict = PardotEmailtemplate2Read()
                        
            if response_dict.get('emailTemplate'):
                if response_dict['emailTemplate']['htmlMessage']:
                    with safe_open_w(AccountName+'/emailtemplates/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(emailtemplate))+"/"+str(int(emailtemplate))+".html") as f:
                        f.write(response_dict['emailTemplate']['htmlMessage'])
                if response_dict['emailTemplate']['textMessage']:
                    with safe_open_w(AccountName+'/emailtemplates/'+str(dt.year)+'/'+str(dt.month)+'/'+str(int(emailtemplate))+"/"+str(int(emailtemplate))+".text") as f:
                        f.write(response_dict['emailTemplate']['textMessage'])
            else:
                print(response_dict)
        else:
            print(emailtemplate)
            print(df.loc[df.email_template_id==emailtemplate].iloc[0]['email_template_id'])


In [ ]:
print("Complete - Records: ",len(response_dict))